# Exercise 5 - Actor Handles

**GOAL:** The goal of this exercise is to show how to pass around actor handles.

Suppose we wish to have multiple tasks invoke methods on the same actor. For example, we may have a single actor that records logging information from a number of tasks. We can achieve this by passing a handle to the actor as an argument into the relevant tasks.

### Concepts for this Exercise - Actor  Handles

First of all, suppose we've created an actor as follows.

```python
@ray.remote
class Actor(object):
    def method(self):
        pass

# Create the actor
actor = Actor.remote()
```

Then we can define a remote function (or another actor) that takes an actor handle as an argument.

```python
@ray.remote
def f(actor):
    # We can invoke methods on the actor.
    x_id = actor.method.remote()
    # We can block and get the results.
    return ray.get(x_id)
```

Then we can invoke the remote function a few times and pass in the actor handle.

```python
# Each of the three tasks created below will invoke methods on the same actor.
f.remote(actor)
f.remote(actor)
f.remote(actor)
```

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from collections import defaultdict
import ray
import time

In [2]:
ray.init(num_cpus=4, include_webui=False, ignore_reinit_error=True)

2020-05-07 18:59:29,835	INFO resource_spec.py:205 -- Starting Ray with 7.42 GiB memory available for workers and up to 3.72 GiB for objects. You can adjust these settings with ray.remote(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '172.18.40.24',
 'redis_address': '172.18.40.24:54081',
 'object_store_address': '/tmp/ray/session_2020-05-07_18-59-29_833922_20356/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-05-07_18-59-29_833922_20356/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-05-07_18-59-29_833922_20356'}

In this exercise, we're going to write some code that runs several "experiments" in parallel and has each experiment log its results to an actor. The driver script can then periodically pull the results from the logging actor.

**EXERCISE:** Turn this `LoggingActor` class into an actor class.

In [3]:
@ray.remote
class LoggingActor(object):
    def __init__(self):
        self.logs = defaultdict(lambda: [])
    
    def log(self, index, message):
        self.logs[index].append(message)
    
    def get_logs(self):
        return dict(self.logs)


assert hasattr(LoggingActor, 'remote'), ('You need to turn LoggingActor into an '
                                         'actor (by using the ray.remote keyword).')

**EXERCISE:** Instantiate the actor.

In [4]:
logging_actor = LoggingActor.remote()

# Some checks to make sure this was done correctly.
assert hasattr(logging_actor, 'get_logs')

Now we define a remote function that runs and pushes its logs to the `LoggingActor`.

**EXERCISE:** Modify this function so that it invokes methods correctly on `logging_actor` (you need to change the way you call the `log` method).

In [5]:
@ray.remote
def run_experiment(experiment_index, logging_actor):
    for i in range(60):
        time.sleep(1)
        # Push a logging message to the actor.
        logging_actor.log.remote(experiment_index, 'On iteration {}'.format(i))

Now we create several tasks that use the logging actor.

In [7]:
experiment_ids = [run_experiment.remote(i, logging_actor) for i in range(3)]
experiment_ids

[ObjectID(1788606d1c6affffffff0100000000c001000000),
 ObjectID(54158c91583effffffff0100000000c001000000),
 ObjectID(b58a281983baffffffff0100000000c001000000)]

While the experiments are running in the background, the driver process (that is, this Jupyter notebook) can query the actor to read the logs.

**EXERCISE:** Modify the code below to dispatch methods to the `LoggingActor`.

In [12]:
logs_id = logging_actor.get_logs.remote()
logs = ray.get(logs_id)
assert isinstance(logs, dict), ("Make sure that you dispatch tasks to the "
                                "actor using the .remote keyword and get the results using ray.get.")
logs

{0: ['On iteration 0',
  'On iteration 1',
  'On iteration 2',
  'On iteration 3',
  'On iteration 4',
  'On iteration 5',
  'On iteration 6',
  'On iteration 7',
  'On iteration 8',
  'On iteration 9',
  'On iteration 0',
  'On iteration 10',
  'On iteration 1',
  'On iteration 11',
  'On iteration 2',
  'On iteration 12',
  'On iteration 3',
  'On iteration 13',
  'On iteration 4',
  'On iteration 14',
  'On iteration 5',
  'On iteration 15',
  'On iteration 6',
  'On iteration 16',
  'On iteration 7',
  'On iteration 17',
  'On iteration 8',
  'On iteration 18',
  'On iteration 9',
  'On iteration 19',
  'On iteration 10',
  'On iteration 20',
  'On iteration 11',
  'On iteration 21',
  'On iteration 12',
  'On iteration 22',
  'On iteration 13',
  'On iteration 23',
  'On iteration 14',
  'On iteration 24',
  'On iteration 15',
  'On iteration 25',
  'On iteration 16',
  'On iteration 26',
  'On iteration 17',
  'On iteration 27',
  'On iteration 18',
  'On iteration 28',
  'On ite

**EXERCISE:** Try running the above box multiple times and see how the results change (while the experiments are still running in the background). You can also try running more of the experiment tasks and see what happens.